# 🧠 Semantica: Enterprise-Grade GraphRAG Pipeline

## 🚀 Overview

This notebook demonstrates the **ultimate** Knowledge Graph orchestration pipeline. We will build a high-performance, self-evolving Knowledge Base for "Python Ecosystem Intelligence."

### 🏗️ Pipeline Architecture

The pipeline is divided into **6 logical phases**:

1.  **Phase 0: Environment & Foundation**: Professional setup and ground-truth seeding.
2.  **Phase 1: Multi-Source Ingestion**: Aggregating data from Web, RSS, and Git.
3.  **Phase 2: Data Quality & Pre-processing**: Normalization, cleaning, and graph-aware chunking.
4.  **Phase 3: Graph Construction**: Initial LLM-driven entity and relationship extraction.
5.  **Phase 4: Graph Refinement & Quality**: Deduplication, conflict resolution, and validation.
6.  **Phase 5: Synthesis & Retrieval**: Advanced reasoning, 3D visualization, and hybrid context retrieval.

---

## 🛠️ Phase 0: Environment & Foundation

We start by setting up the environment and establishing "Ground Truth" data. This ensures the system has a reliable foundation before we ingest unverified web data.

In [ ]:
# 1. Install Dependencies
!pip install -qU semantica networkx matplotlib plotly pandas faiss-cpu tiktoken beautifulsoup4 python-docx pdfplumber

import os
import json
from semantica.core import Semantica, ConfigManager
from semantica.seed import SeedDataManager

# 2. Enterprise Config Definition
config_dict = {
    "project_name": "PythonAI_Mastery",
    "embedding": {"provider": "openai", "model": "text-embedding-3-small"},
    "extraction": {"model": "gpt-4o-mini", "temperature": 0.0},
    "vector_store": {"provider": "faiss", "dimension": 1536},
    "knowledge_graph": {"backend": "networkx", "merge_entities": True, "resolution_strategy": "fuzzy"}
}

config = ConfigManager().load_from_dict(config_dict)
core = Semantica(config=config)

# 3. Seeding Ground Truth (Foundation Graph)
foundation_data = {
    "entities": [
        {"id": "python_org", "name": "Python Software Foundation", "type": "Organization"},
        {"id": "guido_van_rossum", "name": "Guido van Rossum", "type": "Person"}
    ],
    "relationships": [
        {"source": "guido_van_rossum", "target": "python_org", "type": "FOUNDED"}
    ]
}

with open("ground_truth.json", "w") as f: json.dump(foundation_data, f)

seed_manager = SeedDataManager()
seed_manager.register_source("core_info", "json", "ground_truth.json")
foundation_graph = seed_manager.create_foundation_graph()

print(f"✅ Phase 0 Complete. Foundation Graph Seeded with {len(foundation_data['entities'])} Verified Nodes.")

## 📥 Phase 1: Multi-Source Ingestion

We aggregate live data from diverse sources using `semantica.ingest`.

In [ ]:
from semantica.ingest import ingest_web, ingest_feed
from semantica.parse import parse_document

all_content = []

# 1. Web & Docs
web_urls = ["https://www.python.org/about/", "https://realpython.com/"]
for url in web_urls:
    try: all_content.append(ingest_web(url, method="url").text)
    except Exception as e: print(f"Error ingesting {url}: {e}")

# 2. Live RSS Feeds
rss_feeds = ["https://techcrunch.com/feed/", "https://www.wired.com/feed/rss"]
for feed in rss_feeds:
    try:
        feed_data = ingest_feed(feed, method="rss")
        all_content.extend([item.content or item.description for item in feed_data.items[:2]])
    except Exception as e: print(f"Error ingesting feed {feed}: {e}")

# 3. Technical READMEs
repo_files = ["https://raw.githubusercontent.com/psf/requests/main/README.md"]
for file_url in repo_files:
    try: all_content.append(ingest_web(file_url, method="url").text)
    except Exception as e: print(f"Error ingesting {file_url}: {e}")

print(f"✅ Phase 1 Complete. Aggregated {len(all_content)} documents.")

## 🔧 Phase 2: Data Quality & Pre-processing

We ensure the data is clean, structural, and split semantically to preserve entity relationships.

In [ ]:
from semantica.normalize import TextNormalizer, DataCleaner
from semantica.split import EntityAwareChunker

# 1. Normalization & Cleaning
normalizer = TextNormalizer()
cleaner = DataCleaner()

normalized_data = [normalizer.normalize(text) for text in all_content if text]
raw_dataset = [{"text": text, "source_id": i} for i, text in enumerate(normalized_data)]
clean_dataset = cleaner.clean_data(raw_dataset, remove_duplicates=True)

# 2. Graph-Aware Chunking (Ensures entities are not split across chunks)
graph_aware_chunker = EntityAwareChunker(chunk_size=1000, chunk_overlap=200)
all_chunks = []
for doc in clean_dataset:
    all_chunks.extend(graph_aware_chunker.chunk(doc['text']))

print(f"✅ Phase 2 Complete. Generated {len(all_chunks)} high-quality semantic chunks.")

## 🏗️ Phase 3: Graph Construction

We use LLM-driven extraction to build the initial Knowledge Graph.

In [ ]:
from semantica.kg import GraphBuilder

print("Building Knowledge Graph (this may take a moment)...")
gb = GraphBuilder(merge_entities=True)
kg = gb.build(sources=[{"text": str(c.text)} for c in all_chunks[:10]])

print(f"✅ Phase 3 Complete. Entities: {len(kg['entities'])}, Relations: {len(kg['relationships'])}")

## ✨ Phase 4: Graph Refinement & Quality

We refine the raw graph into a production-grade knowledge base.

In [ ]:
from semantica.deduplication import DuplicateDetector, EntityMerger
from semantica.conflicts import ConflictDetector, ConflictResolver
from semantica.kg import GraphValidator

# 1. Deduplication
detector = DuplicateDetector(similarity_threshold=0.85)
duplicates = detector.detect_duplicates(kg.get("entities", []))
if duplicates:
    kg = EntityMerger().merge_duplicates(kg, duplicates)
    print(f"- Deduplicated {len(duplicates)} pairs.")

# 2. Conflict Resolution
conflicts = ConflictDetector().detect_conflicts(kg)
if conflicts:
    kg = ConflictResolver().resolve_conflicts(kg, conflicts, strategy="most_recent")
    print(f"- Resolved {len(conflicts)} conflicts.")

# 3. Final Validation
result = GraphValidator().validate(kg)
status = "✅ Valid" if result.is_valid else f"⚠️ {len(result.issues)} issues"

print(f"✅ Phase 4 Complete. Graph Status: {status}.")

## 🧪 Phase 5: Synthesis, Analytics & Visualization

We apply Graph Analytics and Visualization to derive insights.

In [ ]:
from semantica.kg import CentralityCalculator, CommunityDetector
from semantica.visualization import KGVisualizer
import matplotlib.pyplot as plt

# 1. Analytics
centrality = CentralityCalculator().calculate_degree_centrality(kg)
top_entities = [n['node'] for n in centrality.get("rankings", [])[:3]]

# 2. Visualization
viz = KGVisualizer()
viz.visualize_network(kg, layout="spring", output="static", title="Python Ecosystem Intelligence Graph")
plt.show()

print(f"✅ Phase 5 Complete. Top Entities: {top_entities}")

## 📦 Phase 6: Orchestration & Export

Wrapping everything into a repeatable pipeline and exporting the results.

In [ ]:
from semantica.pipeline import PipelineBuilder
from semantica.export import GraphExporter

# 1. Modular Pipeline Definition
knowledge_pipeline = (
    PipelineBuilder()
    .add_step("ingest", "web_loader")
    .add_step("normalize", "cleaner")
    .add_step("enrich", "kg_builder")
    .build()
)

# 2. Export
GraphExporter().export_to_json(kg, "final_ecosystem_graph.json")

print("✅ Pipeline Orchestration & Export Complete. Project Ready for Deployment.")